In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
train_data=pd.read_excel('Data_Train.xlsx')
train_data.head(5)

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   STORY    7628 non-null   object
 1   SECTION  7628 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 119.3+ KB


In [ ]:
train_data.shape

(7628, 2)

In [ ]:
train_data.groupby('SECTION').describe()

STORY                                                               
        count unique                                                top freq
SECTION                                                                     
0        1686   1673  This story has been published from a wire agen...    4
1        2772   2731  This story has been published from a wire agen...   13
2        1924   1914  We will leave no stone unturned to make the au...    3
3        1246   1233  This story has been published from a wire agen...   11

In [ ]:
train_data.drop_duplicates(inplace=True)
all_punctuations=string.punctuation +'‘’,:”][],' 

In [ ]:
def punc_remover(raw_text):
    no_punct = "".join([i for i in raw_text if i not in all_punctuations])
    return no_punct
    

In [ ]:
def stopword_remover(no_punc_text):
    words= no_punc_text.split()
    no_stp_words = "".join([i for i in words if i not in stopwords.words('english')])
    return no_stp_words
    

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
def lem(words):
    return "".join([lemmer.lemmatize(word,'v')for word in words.split()])

In [ ]:
def text_cleaner(raw):
    cleaned_text = stopword_remover(punc_remover(raw))
    return lem(cleaned_text)

In [ ]:
text_cleaner("Hi!,this is a sample test to the text cleaner method. Remove*@!#special characters%$^* and stopwords. And lemmetizes, go, going - run, ran, running")

'HithissampletesttextcleanermethodRemovespecialcharactersstopwordsAndlemmetizesgogoingrunranrunning'

In [ ]:
train_data['CLEAN_STORY'] = train_data['STORY'].apply(text_cleaner)

In [ ]:
print(train_data.values) 

[['But the most painful was the huge reversal in fee income, unheard of among private sector lenders. Essentially, it means that Yes Bank took it for granted that fees on structured loan deals will be paid and accounted for upfront on its books. As borrowers turned defaulters, the fees tied to these loan deals fell off the cracks. Gill has now vowed to shift to a safer accounting practice of amortizing fee income rather than booking these upfront.\n\n\nGill’s move to mend past ways means that there will be no nasty surprises in the future. This is good news considering that investors love a clean image and loathe uncertainties.\n\n\nBut there is no gain without pain and the promise of a strong and stable balance sheet comes with some sacrifices as well. Investors will have to give up the hopes of phenomenal growth, a promise made by Kapoor.'
  3
  'ButpainfulhugereversalfeeincomeunheardamongprivatesectorlendersEssentiallymeansYesBanktookgrantedfeesstructuredloandealspaidaccountedupfron

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer

In [ ]:
bow_dictionary = CountVectorizer().fit(train_data['CLEAN_STORY'])

In [ ]:
len(bow_dictionary.vocabulary_)

14297

In [ ]:
bow = bow_dictionary.transform(train_data['CLEAN_STORY'])

In [ ]:
print(bow.shape)

(7551, 14297)


In [ ]:
from sklearn.feature_extraction.text import  TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer().fit(bow)

In [ ]:
storyfidf = tfidf_transformer.transform(bow)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
classifier = MultinomialNB().fit(storyfidf,train_data['SECTION'])

In [ ]:
test_data = pd.read_excel('Data_Test.xlsx')

In [ ]:
test_data['CLEAN_STORY'] = test_data['STORY'].apply(text_cleaner)

In [ ]:
print(test_data.values)

[['2019 will see gadgets like gaming smartphones and wearable medical devices lifting the user experience to a whole new level\n\n\nmint-india-wire consumer technologyconsumer technology trends in New Yeartech gadgetsFoldable phonesgaming smartphoneswearable medical devicestechnology\n\n\nNew Delhi: Gadgets have become an integral part of our lives with most of us relying on some form of factor to communicate, commute, work, be informed or entertained. Year 2019 will see some gadgets lifting the user experience to a whole new level. Here’s what we can expect to see:\n\n\nSmartphones with foldable screens: Foldable phones are finally moving from the concept stage to commercial launches. They are made up of organic light-emitting diode (OLED) panels with higher plastic substrates, allowing them to be bent without damage.\n\n\nUS-based display maker Royole Corp’s foldable phone, FlexPai, has already arrived in select markets, while Samsung’s unnamed foldable phone is expected sometime nex

In [ ]:
from sklearn.pipeline import  Pipeline

In [ ]:
pipe = Pipeline([('bow',CountVectorizer()),('tfidf',TfidfTransformer()),('classifier',MultinomialNB())])

In [ ]:
pipe.fit(train_data['CLEAN_STORY'],train_data['SECTION'])

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
test_preds_mnb = pipe.predict(test_data['CLEAN_STORY'])

In [ ]:
pd.DataFrame(test_preds_mnb,columns=['SECTION']).to_excel('predictions.xlsx')